In [1]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

In [2]:
book_List = pd.read_excel("C:/Test1.0.xlsx")
# book_List = pd.read_csv("C:/test.csv")
book_List.head(1)

,ISBN_THIRTEEN_NO,BOOK_TITLE_NM,AUTHR_NM,BOOK_INTRCN_CN,PBLICTE_DE,KDC_NM,AGE_FLAG_NM,SEXDSTN_FLAG_NM,yes24_Sale,yes24_Score,yes24_Price,kyobo_Price,kyobo_Score,kyobo_Sale
0,9788972915782,군주론,저자: 니콜로 마키아벨리 ;김경희,마키아벨리 전공 학자가 번역에 참여한 &lt;군주론&gt;의 이탈리아어 원전 번역판...,2015,340.265,20대,남성,7110,9.3,9000.0,9000,9.63,216


In [3]:
book_List.shape

(30000, 14)

In [4]:
# 주요 컬럼으로 데이터 프레임 생성
col_lst = ['ISBN_THIRTEEN_NO','BOOK_TITLE_NM','AUTHR_NM','BOOK_INTRCN_CN','PBLICTE_DE','KDC_NM','AGE_FLAG_NM','SEXDSTN_FLAG_NM','yes24_Sale','yes24_Score','yes24_Price','kyobo_Price','kyobo_Score','kyobo_Sale']
book_List_df = book_List[col_lst]

In [5]:
# '나이'를 기반으로 'Age_Code' 컬럼 생성
def map_age_code(AGE_FLAG_NM):
    if AGE_FLAG_NM in ['영유아', '유아']:
        return 1
    elif AGE_FLAG_NM == '초등':
        return 2
    elif AGE_FLAG_NM == '청소년':
        return 3
    elif AGE_FLAG_NM == '20대':
        return 4
    elif AGE_FLAG_NM == '30대':
        return 5
    elif AGE_FLAG_NM == '40대':
        return 6
    elif AGE_FLAG_NM == '50대':
        return 7
    elif AGE_FLAG_NM == '60대이상':
        return 8
    else:
        return None  # 예외 처리: 다른 값이 들어오면 None 반환

# 'Age_Code' 컬럼 추가
book_List_df['Age_Code'] = book_List_df['AGE_FLAG_NM'].apply(map_age_code)


# '성별'를 기반으로 'Sex_Code' 컬럼 생성
def map_sex_code(SEXDSTN_FLAG_NM):
    if SEXDSTN_FLAG_NM == '남성':
        return 1
    elif SEXDSTN_FLAG_NM == '여성':
        return 2
    else:
        return None  # 예외 처리: 다른 값이 들어오면 None 반환

# 'Age_Code' 컬럼 추가
book_List_df['Sex_Code'] = book_List_df['SEXDSTN_FLAG_NM'].apply(map_sex_code)


In [6]:
# [교보] [Yes24] 각각에서 크롤링한 데이터를 평균치를 구해야하나??

# 일단 가격 
for index, row in book_List_df.iterrows():
    if row['kyobo_Price'] <= 0 or (row['kyobo_Price'] == 0 and row['yes24_Price'] == 0):
        book_List_df.at[index, 'avg_price'] = row['yes24_Price']
    else:
        avg_price = (row['yes24_Price'] + row['kyobo_Price']) / 2
        book_List_df.at[index, 'avg_price'] = avg_price


# 별점 * 별점 확인결과 교보별점 데이터에만 0인 데이터가 있음
for index, row in book_List_df.iterrows():
    if row['kyobo_Score'] == 0:
        book_List_df.at[index, 'avg_Score'] = row['yes24_Score']
    else:
        avg_Score = (row['yes24_Score'] + row['kyobo_Score']) / 2
        book_List_df.at[index, 'avg_Score'] = avg_Score

# 리뷰count는 평균을 구할 필요는 없어보임 
for index, row in book_List_df.iterrows():
    Sale_Sum = (row['yes24_Sale'] + row['kyobo_Sale'])
    book_List_df.at[index, 'Sale_Sum'] = Sale_Sum

In [7]:
# 주요 컬럼으로 데이터 프레임 생성
# New column = Age_Code, avg_price, avg_Score, Sale_Sum
col_lst = ['ISBN_THIRTEEN_NO','BOOK_TITLE_NM','AUTHR_NM','Age_Code','Sex_Code','BOOK_INTRCN_CN','PBLICTE_DE','KDC_NM','avg_price','avg_Score','Sale_Sum']
book_List_df_Avg = book_List_df[col_lst]

book_List_df_Avg.head(1)

,ISBN_THIRTEEN_NO,BOOK_TITLE_NM,AUTHR_NM,Age_Code,Sex_Code,BOOK_INTRCN_CN,PBLICTE_DE,KDC_NM,avg_price,avg_Score,Sale_Sum
0,9788972915782,군주론,저자: 니콜로 마키아벨리 ;김경희,4.0,1.0,마키아벨리 전공 학자가 번역에 참여한 &lt;군주론&gt;의 이탈리아어 원전 번역판...,2015,340.265,9000.0,9.465,7326.0


In [9]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from konlpy.tag import Okt
import re

# 장르별
book_List_df_Avg['genres_literal'] = book_List_df_Avg['KDC_NM'].astype(str)

# 책 재목
book_List_df_Avg['title_literal'] = book_List_df_Avg['BOOK_TITLE_NM'].astype(str)

# 책 줄거리 내용 
# book_List_df_Avg['book_intrcn_literal'] = book_List_df_Avg['BOOK_INTRCN_CN'].astype(str)
# 이 코드는 주어진 텍스트 데이터에 대해 불용어 제거, 특수 문자 제거, 형태소 분석 등을 수행하여 텍스트 데이터를 정제하는 과정 포함
# 한글 형태소 분석기 초기화
okt = Okt()

def preprocess_text(text):
    if pd.isnull(text):
        return ''  # Return empty string for NaN values
    else:
        # 불용어 제거
        stop_words = ['는', '은', '이', '가', '을', '를', '의', '에서', '에게', '에', '으로', '로']
        words = [word for word in okt.morphs(text) if word not in stop_words]

        # 특수 문자 제거
        words = [re.sub(r'[^a-zA-Z가-힣0-9]', ' ', word) for word in words]

        # 형태소 분석
        words = [okt.morphs(word) for word in words]

        # 공백 제거 및 단어 재조합
        words = [' '.join(word) for word in words]

        return ' '.join(words)

# 텍스트 데이터에 전처리 적용
book_List_df_Avg['book_intrcn_literal'] = book_List_df_Avg['BOOK_INTRCN_CN'].apply(preprocess_text)

# Concatenate 'title_literal' with 'genres_literal' and 'book_intrcn_literal'
book_List_df_Avg['combined_features'] = (
    book_List_df_Avg['genres_literal'] + ' ' +
    book_List_df_Avg['book_intrcn_literal'] + ' ' +
    book_List_df_Avg['title_literal']
)

# CountVectorizer를 이용한 장르 유사도 계산
count_vect = CountVectorizer(min_df=1, ngram_range=(1, 2))
genre_mat_dense = count_vect.fit_transform(book_List_df_Avg['combined_features'])

# Convert dense matrix to sparse matrix
genre_mat = csr_matrix(genre_mat_dense)

# 코사인 유사도 계산 (희소 행렬 직접 사용)
genre_sim = cosine_similarity(genre_mat, dense_output=False)

# 선택한 책에 대한 유사도만을 추출할 책의 인덱스 설정
title_index = 0

# 선택한 책 대한 유사도만을 추출
selected_book_similarity = genre_sim[title_index, :].reshape(-1, 1)

# Choose a lower dimensionality
n_components = 30

# Apply TruncatedSVD to reduce dimensionality
svd = TruncatedSVD(n_components=n_components)
genre_mat_reduced = svd.fit_transform(genre_mat)

# Compute cosine similarity on the reduced matrix
genre_sim_reduced = cosine_similarity(genre_mat_reduced, dense_output=False)

# 선택한 책에 대한 유사도만을 추출
selected_book_similarity_reduced = genre_sim_reduced[title_index, :].reshape(-1, 1)

C:\Users\user\AppData\Local\Temp\ipykernel_24076\168039470.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book_List_df_Avg['genres_literal'] = book_List_df_Avg['KDC_NM'].astype(str)
C:\Users\user\AppData\Local\Temp\ipykernel_24076\168039470.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book_List_df_Avg['title_literal'] = book_List_df_Avg['BOOK_TITLE_NM'].astype(str)
C:\Users\user\AppData\Local\Temp\ipykernel_24076\168039470.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

def calculate_intrcn_similarity(df):
    df['BOOK_INTRCN_CN'].fillna('', inplace=True)
    count_vectorizer = CountVectorizer(stop_words='english')
    intrcn_matrix = count_vectorizer.fit_transform(df['BOOK_INTRCN_CN'])
    intrcn_sim = cosine_similarity(intrcn_matrix, intrcn_matrix)
    return intrcn_sim

def normalize_feature(feature):
    return (feature - feature.min()) / (feature.max() - feature.min())

def find_sim_movie(df, sim_matrix, title_isbn, top_n):
    title_book = df[df['ISBN_THIRTEEN_NO'] == title_isbn]

    if title_book.empty:
        print(f"No book found with ISBN {title_isbn}")
        return pd.DataFrame()

    title_index = title_book.index.values[0]
    
    df["similarity"] = sim_matrix[title_index].toarray().flatten()

    df = df[df["avg_Score"] != -1]

    df["normalized_score"] = normalize_feature(df["avg_Score"])
    df["normalized_price"] = 1 - normalize_feature(df["avg_price"])
    df["normalized_Sale"] = normalize_feature(df["Sale_Sum"])

    intrcn_sim = calculate_intrcn_similarity(df)
    df["intrcn_similarity"] = intrcn_sim[title_index]

    df["combined_similarity"] = (
        df["similarity"] +
        0.1 * df["normalized_score"] + # 별점
        0.1 * df["normalized_price"] + # 가격
        0.3 * df["normalized_Sale"] +  # 리뷰 수
        0.5 * df["intrcn_similarity"] # 제목, 장르, 책줄거리
    )

    temp = df.sort_values(by=["combined_similarity", "Sale_Sum"], ascending=[False, False])
    temp = temp.drop_duplicates(subset=['ISBN_THIRTEEN_NO','BOOK_TITLE_NM','Sex_Code','AUTHR_NM','BOOK_INTRCN_CN','PBLICTE_DE','KDC_NM','avg_price','avg_Score','Sale_Sum'])

    # 해당 문자열 자르기는 추출된 값을 보기 좋게하기위해 작성된 코드 
    # 지워도 무관함
    temp['BOOK_TITLE_NM'] = temp['BOOK_TITLE_NM'].str[:15]
    temp['AUTHR_NM'] = temp['AUTHR_NM'].str[:25]
    temp['BOOK_INTRCN_CN'] = temp['BOOK_INTRCN_CN'].str[:15]

    final_index = temp.index.values[:min(top_n, len(temp))]

    return temp.iloc[:top_n] if len(temp) >= top_n else temp

similar_books = find_sim_movie(book_List_df.copy(), genre_sim, 9791189001261, 15)


In [13]:
from tabulate import tabulate

# 결과 출력
print("남성에게 추천하는 책:")
print(tabulate(similar_books[similar_books['Sex_Code'] == 1][['ISBN_THIRTEEN_NO','BOOK_TITLE_NM','Sex_Code','AUTHR_NM','KDC_NM','avg_price','avg_Score','Sale_Sum','intrcn_similarity']],
               headers='keys', tablefmt='psql', showindex=False, numalign="center", stralign="center", colalign=("center",)*7))

print("\n여성에게 추천하는 책:")
print(tabulate(similar_books[similar_books['Sex_Code'] == 2][['ISBN_THIRTEEN_NO','BOOK_TITLE_NM','Sex_Code','AUTHR_NM','KDC_NM','avg_price','avg_Score','Sale_Sum','intrcn_similarity']],
               headers='keys', tablefmt='psql', showindex=False, numalign="center", stralign="center", colalign=("center",)*7))


남성에게 추천하는 책:
+--------------------+------------------------+------------+-------------------------+----------+-------------+-------------+------------+---------------------+
|  ISBN_THIRTEEN_NO  |     BOOK_TITLE_NM      |  Sex_Code  |        AUTHR_NM         |  KDC_NM  |  avg_price  |  avg_Score  |  Sale_Sum  |  intrcn_similarity  |
|--------------------+------------------------+------------+-------------------------+----------+-------------+-------------+------------+---------------------|
|   9791196893118    | 설민석의 세계사 대모험 |     1      | 글: 설민석,그림: 박성일 |   909    |    10800    |    9.74     |    2636    |      0.242871       |
|   9791196893118    | 설민석의 세계사 대모험 |     1      | 글: 설민석,그림: 박성일 |   909    |    10800    |    9.74     |    2636    |      0.242871       |
|   9791196893118    | 설민석의 세계사 대모험 |     1      | 글: 설민석,그림: 박성일 |   909    |    10800    |    9.74     |    2636    |      0.242871       |
|   9791196893132    | 설민석의 세계사 대모험 |     1      | 글: 설민석,그림: 박성일 |   909    |

In [11]:

#
#import pymysql
#import pandas as pd

# similar_movies 데이터프레임의 인덱스 초기화
#similar_movies_reset = similar_movies.reset_index(drop=True)

# 'ISBN_THIRTEEN_NO' 열의 값을 가져와서 SQL 쿼리에 사용하기
#isbn_values = similar_movies_reset['ISBN_THIRTEEN_NO'].values
#isbn_str = ",".join([str(isbn) for isbn in isbn_values])

#conn = pymysql.connect(host='127.0.0.1',
#                       user='root',
#                       password='1234',
#                       db='Test_one',
#                       charset='utf8')
#cursor = conn.cursor()

#try:
    # SELECT 쿼리 실행
#    query = f"SELECT SEXDSTN_FLAG_NM, COUNT(*) AS Count FROM Test_one.Book_Sex_Code_v2 WHERE ISBN_THIRTEEN_NO IN ({isbn_str}) GROUP BY SEXDSTN_FLAG_NM;"
#    cursor.execute(query)

    # 결과 가져오기
#    result = cursor.fetchall()

    # 결과를 DataFrame으로 변환
#    df_result = pd.DataFrame(result, columns=[desc[0] for desc in cursor.description])

    # 출력
#    print(df_result)

#except Exception as e:
#    print(f"Error: {e}")

#finally:
    # 연결 및 커서 닫기
#    cursor.close()
#    conn.close()